In [1]:
from qiskit import *
from qiskit.tools.visualization import plot_histogram
%matplotlib inline

### Superdense coding

Trasmit the 2 bits by sending just one qubit to another. Both qubits are entangled

In [ ]:
circuit = QuantumCircuit(2)

# Prepare the Bell state and distribute qubits
circuit.h(0)
circuit.cx(0, 1)

circuit.barrier()


# Olivia encodes her message and transmits her qubits to Barron
match message := '00':
    # apply identity operator
    case '00':
        circuit.id(0)
    case '01':
        circuit.z(0)
    case '10':
        circuit.x(0)
    case '11':
        circuit.z(0)
        circuit.x(0)

circuit.barrier()

Barron decodes the message by entangling them

In [ ]:
circuit.cx(0, 1)
circuit.h(0)

Measuring the qubits to read Olivia's message

In [ ]:
circuit.measure_all()
circuit.draw(output='mpl')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend=simulator).result()
plot_histogram(result.get_counts())

### Quantum teleportation

Requires source qubit and 2 entangled qubits for transportation

In [6]:
from qiskit import *
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram
from math import pi
%matplotlib inline

In [ ]:
source = QuantumRegister(1, name='sourse')
q1 = QuantumRegister(1, name='q1')
destination = QuantumRegister(1, name='destination')

crz = ClassicalRegister(1, name="crz")
crx = ClassicalRegister(1, name="crx")

circuit = QuantumCircuit(source, q1, destination, crz, crx)

# initialize source qubit state
circuit.x(0)
circuit.barrier()

# create State Bell pair
circuit.h(q1)
circuit.cx(q1, destination)

# measure the state
circuit.cx(source, q1)
circuit.h(source)
circuit.barrier()
circuit.measure(source, crz)
circuit.measure(q1, crx)
circuit.barrier()

# transform destination the qubit based on the measured values
circuit.x(destination).c_if(crx, 1)
circuit.z(destination).c_if(crz, 1)

circuit.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
statevector = execute(circuit, backend=backend).result().get_statevector()
plot_bloch_multivector(statevector)

In [ ]:
backend = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend = backend).result()
plot_histogram(result.get_counts())

Due to Deferred Measurement Principle we should not apply any operations after any measurements. So here is the deferred measurement approach

In [ ]:
circuit = QuantumCircuit(3, 1)

# Initialize source qubit
circuit.x(0)
circuit.barrier()

# Bell State pair
circuit.h(1)
circuit.cx(1, 2)
circuit.barrier()

# Bell State measurements
circuit.cx(0, 1)
circuit.h(0)
circuit.barrier()

# Conditional operations on destination qubit
circuit.cx(1,2)
circuit.cz(0,2)
circuit.barrier()

circuit.measure(2, 0)
circuit.draw(output='mpl')

In [ ]:
result = execute(circuit, backend=Aer.get_backend('qasm_simulator')).result()
plot_histogram(result.get_counts())